In [6]:
#распознавание рукописного текста
import numpy as np
from keras.preprocessing import image

from keras.datasets import mnist#-классификация рукописных цифр
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD #SGD - стохастический градиентный спуск
from keras.layers import Flatten, Activation
from keras.models import Sequential#модель нейронной сети слои которой соединены друг с другом
from keras.layers import Dense#соединение всех нейронов предыдущего уровня со всми нейронами следующего уровня
from keras.utils import np_utils#утилиты для работы с массивами
from keras.models import model_from_json

# Размер изображения
img_rows, img_cols = 28, 28



# Устанавливаем seed для повторяемости результатов
np.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Преобразование размерности изображений
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# Создаем последовательную модель
model = Sequential()

# Первый сверточный слой
model.add(Conv2D(75, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))

# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой регуляризации Dropout
model.add(Dropout(0.2))#Dropout - предотвращает переобучение
#0.2 - нейрон будет отключаться с вероятностью 20%
#Оставшиеся нейроны обучаются распознавать признаки без участия соседних

# Второй сверточный слой
model.add(Conv2D(100, (5, 5), activation='relu'))

# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой регуляризации Dropout
model.add(Dropout(0.2))#Dropout - предотвращает переобучение
#0.2 - нейрон будет отключаться с вероятностью 20%
#Оставшиеся нейроны обучаются распознавать признаки без участия соседних

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())

# Полносвязный слой для классификации
model.add(Dense(500, activation='relu'))

# Слой регуляризации Dropout
model.add(Dropout(0.5))#Dropout - предотвращает переобучение
#0.2 - нейрон будет отключаться с вероятностью 20%
#Оставшиеся нейроны обучаются распознавать признаки без участия соседних

# Выходной полносвязный слой
model.add(Dense(10, activation='softmax'))

# Компилируем модель
#SGD - стохастический градиентный спуск
model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

print(model.summary())

# Обучаем сеть
# размер минивыборки = 200 элементов, epochs = количество обучений(лучший результат при 150)
#20% - проверочная выборка
model.fit(X_train, Y_train, batch_size=200, epochs=120, validation_split=0.2, verbose=2)

# Оцениваем качество обучения сети на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

#Сохраняем обученную нейронную сеть в файл

# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("mnistreal_model.json", "w")
json_file.write(model_json)
json_file.close()
#Для сохранения данных о весах сети Keras использует формат HDF5
model.save_weights("mnistreal_model.h5")

#Загружаем обученную нейронную сеть из файла

# Загружаем данные об архитектуре сети из файла json
json_file = open("mnistreal_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights("mnistreal_model.h5")

#Перед использованием модели, ее обязательно нужно скомпилировать:

# Компилируем модель
loaded_model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
# Проверяем модель на тестовых данных
scores = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("Точность модели на тестовых данных: %.2f%%" % (scores[1]*100))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 24, 24, 75)        1950      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 75)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 12, 12, 75)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 100)         187600    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 100)         0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 4, 4, 100)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
__________

Epoch 76/150
 - 198s - loss: 0.0540 - acc: 0.9834 - val_loss: 0.0416 - val_acc: 0.9880
Epoch 77/150
 - 199s - loss: 0.0531 - acc: 0.9835 - val_loss: 0.0423 - val_acc: 0.9879
Epoch 78/150
 - 198s - loss: 0.0528 - acc: 0.9836 - val_loss: 0.0416 - val_acc: 0.9879
Epoch 79/150
 - 198s - loss: 0.0525 - acc: 0.9830 - val_loss: 0.0410 - val_acc: 0.9878
Epoch 80/150
 - 198s - loss: 0.0537 - acc: 0.9829 - val_loss: 0.0416 - val_acc: 0.9878
Epoch 81/150
 - 199s - loss: 0.0528 - acc: 0.9836 - val_loss: 0.0407 - val_acc: 0.9883
Epoch 82/150
 - 198s - loss: 0.0515 - acc: 0.9837 - val_loss: 0.0403 - val_acc: 0.9883
Epoch 83/150
 - 198s - loss: 0.0510 - acc: 0.9839 - val_loss: 0.0401 - val_acc: 0.9882
Epoch 84/150
 - 199s - loss: 0.0512 - acc: 0.9846 - val_loss: 0.0399 - val_acc: 0.9883
Epoch 85/150
 - 198s - loss: 0.0489 - acc: 0.9842 - val_loss: 0.0396 - val_acc: 0.9886
Epoch 86/150
 - 198s - loss: 0.0500 - acc: 0.9843 - val_loss: 0.0400 - val_acc: 0.9882
Epoch 87/150
 - 199s - loss: 0.0485 - acc: 